In [1]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
import pandas as pd

In [2]:
urls = ["https://goias.gov.br/social/perguntas-e-respostas-frequentes-aprendiz-do-futuro/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-dignidade/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-maes-de-goias/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-goias-por-elas/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-sistema-socioeducativo/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-auxilios-agua-e-energia-pao-e-leite/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-carteira-de-identificacao-do-autista2/",
        "https://goias.gov.br/social/bolsa-familia/",
        "https://goias.gov.br/social/cadastro-unico-cadunico/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-passaporte-intermunicipal-da-pessoa-idosa/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-passe-livre-intermunicipal-da-pessoa-com-deficiencia/",
        "https://goias.gov.br/social/perguntas-e-respostas-frequentes-centro-de-referencia-estadual-da-igualdade-crei/"]

inicio = """Categoria  

Perguntas e Respostas Frequentes"""

fim = """


 











 









 













 

 





 Governo na palma da mão"""

complemento_texto = " O que devo fazer?"

In [ ]:
textos_brutos = []
textos_conteudo = []
textos_process = []

for url in urls:
    textos = WebBaseLoader(url)
    textos_brutos = textos.load()
    textos_conteudo.append(textos_brutos[0].page_content)

In [ ]:

len(" O que devo fazer?")

In [ ]:
tamanho_inicio = len(inicio)
tamanho_final = len(fim)
tamanho_complemento = len(complemento_texto)

for respostas_ind in textos_conteudo:
    for j in range(len(respostas_ind) - tamanho_inicio):
        if inicio in respostas_ind[j:j + tamanho_inicio]:
            começo_frase = j + tamanho_inicio
        elif fim in respostas_ind[j:j+tamanho_final]:
            textos_process.append(respostas_ind[começo_frase + 6 : j])

In [ ]:
print(textos_process[11])

In [ ]:
textos_split = []
questions = []
cont = 0
contador = 0
perguntas = []
answers = []
respostas = []

In [ ]:
for texto in textos_process:
    for i, caractere in enumerate(texto):
        contador = 0
        aux = i 
        if "." in caractere and contador != 1:
            if complemento_texto in texto[i: i + tamanho_complemento  + 1]:
                print("passei por aqui 2")
                complemento = i + 18
                while contador != 1:
                    if "." in texto[aux]:
                        print("passei por aqui")
                        questions.append(texto[aux : complemento])
                        contador = 1
                    aux = i - 1
                    
        if "?" in caractere:
            pergunta = ""
            interrogacao = i
            cont = 1  # Inicializa cont
            contador = 0
            while contador != 1:
                if texto[i - cont] == "." or texto[i - cont] == ";" or texto[i - cont] == "\n" or (i - cont) == 0 :
                    j = i - cont
                    contador = 1
                cont = cont + 1
            cont = 0
            while j + cont <= i:
                pergunta = pergunta + texto[j + cont]
                cont = cont + 1
            questions.append(pergunta)

In [ ]:
questions = [caractere.replace("\n", "") for caractere in questions]
questions = [caractere.replace(". ", "") for caractere in questions]

In [ ]:
len(questions)
cont = 0
print(questions[1])

In [ ]:
while cont < len(questions):
    for texto in textos_process:
        for i, caractere in enumerate(texto):
            if cont < len(questions) - 1:
                if questions[cont] in texto[i: i + len(questions[cont])]:
                    contador = 0
                    inicio_respostas = i + len(questions[cont]) + 1
                    if cont + 1 < len(questions):
                        j = inicio_respostas
                        for j in range (len(texto) - len(questions[cont + 1])):
                            if questions[cont + 1] in texto[j: j + len(questions[cont + 1])]:
                                print("passei por aqui 1")
                                answers.append(texto[inicio_respostas : j - 1])
                                cont = cont + 1
                                break
                            if questions[cont + 1] not in texto[j: j + len(questions[cont + 1])] and j + len(questions[cont + 1]) == len(texto) - 1:
                                print("passei por aqui 2")
                                answers.append(texto[i + len(questions[cont]) :])
                                cont = cont + 1
                                break 
            elif cont == len(questions) - 1:
                answers.append(texto[i:])
                cont = cont + 1

In [ ]:
print(answers[0])

In [ ]:
questions

In [ ]:
all_data =[]


In [ ]:
for item in range(len(questions)):
    question = questions[item]
    answer = answers[item]
    all_data.append({'question': question, 'answer': answer})

In [ ]:
data_df = pd.DataFrame(all_data)
data_df.to_csv('Data/rag_doc.csv', index=False)